In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [ ]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split as tts

import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D, Conv2D, \
                                    MaxPool2D, ZeroPadding2D, BatchNormalization, Input, DepthwiseConv2D, Add, LeakyReLU, ReLU
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score


print('tensorflow version:', tf.__version__)

tensorflow version: 2.8.2


## data load

In [ ]:
train_test = np.load('/content/mydrive/MyDrive/AIFFELTHON/kakao_arena/mel_preprocessed/f_list_0_10000.npy')

In [ ]:
train_test.shape

(9986, 48, 1876, 1)

In [ ]:
x_train, x_test = tts(train_test, test_size=0.2)

print(x_train.shape)
print(x_test.shape)

(7988, 48, 1876, 1)
(1998, 48, 1876, 1)


In [ ]:
test_data = np.random.random((9986, 6))
test_data

array([[0.42310607, 0.26300496, 0.89332795, 0.94022019, 0.71350086,
        0.76172377],
       [0.62905362, 0.60996775, 0.78501363, 0.27493135, 0.42144897,
        0.96914037],
       [0.69916421, 0.80115274, 0.884296  , 0.49890113, 0.63903061,
        0.32772005],
       ...,
       [0.07832577, 0.90686169, 0.01610637, 0.08120896, 0.98699019,
        0.55387039],
       [0.93225606, 0.10934125, 0.93651069, 0.66610862, 0.49640238,
        0.74304384],
       [0.66903624, 0.13626548, 0.35321746, 0.28957911, 0.95822061,
        0.30207084]])

In [ ]:
'''
test_data에는 라벨 데이터가 들어가야 함
라벨 데이터의 shape는 (N, 1, 6) 형태
'''
y_train, y_test = tts(test_data, test_size=0.2)

print(y_train.shape)
print(y_test.shape)

(7988, 6)
(1998, 6)


## simple cnn

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 1)),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])
# model.summary() # 신경망의 구조 확인

In [ ]:
'''
input size = (N, 48, 1876, 1)

1876 = 2*2*7*67
48 = 2*2*2*2*3
'''

embedding_dim = 48 # 임베딩 벡터의 차원 // 여기서는 hz의 채널 수
kernel_size = 4
drop_ratio = 0.3
hidden_units = 512 # 뉴런의 수

def build_cnn():
    model = Sequential()
    model.add(Conv2D(16, kernel_size, padding='valid', activation='relu', input_shape=(48, 1876, 1)))
    model.add(MaxPool2D(kernel_size))
    model.add(Conv2D(32, kernel_size, padding='valid', activation='relu'))
    model.add(MaxPool2D(kernel_size))
    model.add(Flatten())
    model.add(Dense(hidden_units, activation='relu'))
    model.add(Dropout(drop_ratio))
    model.add(Dense(6, activation='softmax'))
    
    return model

model = build_cnn()
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 45, 1873, 16)      272       
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 11, 468, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 8, 465, 32)        8224      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 2, 116, 32)       0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 7424)              0         
                                                                 
 dense_17 (Dense)            (None, 512)             

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test), callbacks=[es, mc])

Epoch 1/20
250/250 [==============================] - ETA: 0s - loss: 81010560.0000 - acc: 0.1670
Epoch 1: val_acc improved from -inf to 0.16817, saving model to best_model.h5
250/250 [==============================] - 7s 22ms/step - loss: 81010560.0000 - acc: 0.1670 - val_loss: 43301796.0000 - val_acc: 0.1682
Epoch 2/20
250/250 [==============================] - ETA: 0s - loss: 2045210880.0000 - acc: 0.1694
Epoch 2: val_acc improved from 0.16817 to 0.17267, saving model to best_model.h5
250/250 [==============================] - 5s 19ms/step - loss: 2045210880.0000 - acc: 0.1694 - val_loss: 751107904.0000 - val_acc: 0.1727
Epoch 3/20
249/250 [============================>.] - ETA: 0s - loss: 10148216832.0000 - acc: 0.1610
Epoch 3: val_acc did not improve from 0.17267
250/250 [==============================] - 5s 18ms/step - loss: 10171256832.0000 - acc: 0.1610 - val_loss: 2064603904.0000 - val_acc: 0.1682
Epoch 4/20
247/250 [============================>.] - ETA: 0s - loss: 2732450201